# Imports & Utils

In [110]:
import os
from PIL import Image
import numpy as np
import struct
import random
from collections import Counter
import pickle

In [28]:
# fib[i] stores i+2th fib number
# limiter: fib[12] (stores the 14th Fibonacci No.) i.e. 377, we only need to code values upto 255
N = 12
fib = [0 for _ in range(N+1)]
fib[0] = 1
fib[1] = 2

def calFib(N):
    if fib[N] != 0:
        return fib[N]
    fib[N] = calFib(N-1) + calFib(N-2)
    return fib[N]

calFib(N)
print(fib)

[1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377]


# Encoder & Decoder Functions

- In Traditional Image Storage (8-bit per Pixel).
- In Fibonacci encoding, we use variable length encoding for example: 10 (100101), 50(101001001), 101(1000010101).
- An 1 is appended at the end to separate the data.
- Fibonacci encoding uses an average of 10.625 bits per pixel for values 0-255, which is higher than the fixed 8-bit representation.
- If an image has a wide range of intensities or bright pixels, Fibonacci encoding may increase storage size.

In [3]:
# Return index of the largest fibonacci number smaller than or equal to n
def largestFiboLessOrEqual(n):
    i = 0
    while fib[i] <= n:
        i += 1
    # loop stopped when fib[i] became larger
    return i - 1

In [4]:
# returns a list of string of encoded pixels
def fib_encoder_v1(pixels_1D):
    encoded_1D = []
    for pixel in pixels_1D:
        i = largestFiboLessOrEqual(pixel)
        encoded_pixel = ""

        while pixel:
            encoded_pixel += '1'
            pixel -= fib[i]
            i -= 1

            # not needed bits
            while (i >= 0 and fib[i] > pixel):
                encoded_pixel += '0'
                i -= 1

        # '1' bit padding
        reversed_encoded_pixel = encoded_pixel[::-1]
        reversed_encoded_pixel += '1'
        encoded_1D.append(reversed_encoded_pixel)
    return encoded_1D

In [5]:
def fib_decoder_v1(encoded_strings_1D):
    decoded_1D = []

    for encoded_string in encoded_strings_1D:
        pixel = 0
        # ignore the padding
        for i in range(len(encoded_string) - 1):
            if int(encoded_string[i]) == 1:
                # work on reverse
                index = i
                pixel += fib[index]
        decoded_1D.append(pixel)

    return decoded_1D

# Helper Functions

In [91]:
def convert_to_int_list(string_list):
    # Concatenate all binary strings
    binary_string = ''.join(string_list)

    # Pad with '0's to make the length a multiple of 8
    while len(binary_string) % 8 != 0:
        binary_string += '0'

    # Split into 8-bit chunks and convert to integers
    int_array = [int(binary_string[i:i+8], 2) for i in range(0, len(binary_string), 8)]

    return int_array

In [92]:
print(convert_to_int_list(fib_encoder_v1([1, 1, 1, 1, 1])))

[255, 192]


In [93]:
def convert_to_binary_string(int_array):
    # Convert each integer to an 8-bit binary string
    binary_list = [format(num, '08b') for num in int_array]

    # Concatenate all binary strings into one
    binary_string = ''.join(binary_list)

    # Remove any trailing zero-padding
    binary_string = binary_string.rstrip('0')

    return binary_string

In [94]:
print(convert_to_binary_string([8, 10]))

000010000000101


In [95]:
def extract_fib_encoded_strings(concatenated_string):
    fib_encoded_strings = []
    start_index = 0
    length = len(concatenated_string)

    i = 0
    while i < length - 1:
        if concatenated_string[i] == '1' and concatenated_string[i + 1] == '1':  # Detect Fibonacci termination "11"
            fib_encoded_strings.append(concatenated_string[start_index:i + 2])
            start_index = i + 2
            i += 1
        i += 1

    return fib_encoded_strings

In [96]:
print(extract_fib_encoded_strings("1111"))

['11', '11']


In [97]:
print((fib_decoder_v1(extract_fib_encoded_strings(convert_to_binary_string(convert_to_int_list(fib_encoder_v1(list(range(1, 256)))))))))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 22

# API

In [118]:
def fib_interface_image_compression(image_file, compressed_file, version=1):
    if version not in [1]:
        raise ValueError("Invalid version!")

    image = Image.open(image_file)
    image_data = np.array(image)
    height, width, *channels = image_data.shape
    image_array_1D = image_data.flatten()
    incremented_image_array_1D = [x + 1 if x < 255 else x for x in image_array_1D]

    frequency = Counter(incremented_image_array_1D)
    # Sort by frequency (highest to lowest)
    sorted_frequency = sorted(frequency.items(), key=lambda x: x[1], reverse=True)
    # Create a rank map based on frequency
    rank_map = {number: rank + 1 for rank, (number, _) in enumerate(sorted_frequency)}
    reverse_rank_map = {rank: number for number, rank in rank_map.items()}
    # Replace original pixel value with their ranks
    ranked_incremented_image_array_1D = [rank_map[number] for number in incremented_image_array_1D]

    encoder_functions = {1: fib_encoder_v1}
    compressed_string_list = encoder_functions[version](ranked_incremented_image_array_1D)
    compressed_int_list = convert_to_int_list(compressed_string_list)

    # save compressed data
    with open(compressed_file, "wb") as f:
        shape_str = f"{height}x{width}" + (f"x{channels[0]}" if channels else "")
        f.write(f"{shape_str}\n".encode())
        f.write(bytearray(compressed_int_list))

    with open("fib_rank_map.bin", "wb") as f:
        pickle.dump(reverse_rank_map, f)

In [119]:
def fib_interface_image_decompression(compressed_file, image_file, version=1):
    if version not in [1]:
        raise ValueError("Invalid version!")

    with open(compressed_file, "rb") as f:
        shape_line = f.readline().decode().strip()
        shape = tuple(map(int, shape_line.split('x')))
        compressed_list = list(f.read())

    with open("fib_rank_map.bin", "rb") as f:
        rank_map = pickle.load(f)

    compressed_concat_binary_string = convert_to_binary_string(compressed_list)
    compressed_string_list = extract_fib_encoded_strings(compressed_concat_binary_string)

    decoder_functions = {1: fib_decoder_v1}
    decompressed_1D = decoder_functions[version](compressed_string_list)

    original_decompressed_1D = [rank_map[rank] for rank in decompressed_1D]

    decremented_decompressed_1D = [x - 1 if random.random() < 0.5 else x for x in original_decompressed_1D]
    decompressed_2D = np.array( decremented_decompressed_1D, dtype=np.uint8).reshape(shape)

    image = Image.fromarray(decompressed_2D)
    image.save(image_file)

# Print Functions

In [121]:
def file_details(file_path):
    if os.path.exists(file_path):
        print(f"File Name: {os.path.basename(file_path)}")
        print(f"File Size: {os.path.getsize(file_path)} bytes")
    else:
        print("File does not exist.")

In [122]:
def calculate_compression_factor(original_file, compressed_file):
    original_size = os.path.getsize(original_file)
    rank_file_size = os.path.getsize("fib_rank_map.bin")
    compressed_size = os.path.getsize(compressed_file)

    if compressed_size == 0:
        return float('inf'), 100.0

    compression_factor = (original_size + rank_file_size) / compressed_size
    compression_ratio = (1 - (compressed_size / (original_size + rank_file_size))) * 100

    return compression_factor, compression_ratio

In [123]:
def show(image_file):
    image = Image.open(image_file)
    image.show(image_file)

# Interface

In [131]:
image_file = "Color500.bmp"
file_details(image_file)

compression_interfaces = {1: fib_interface_image_compression}
decompression_interfaces = {1: fib_interface_image_decompression}

interface = 1
encoding_version = 1

compressed_file = "compressed_file.bin"
compression_interfaces[interface](image_file, compressed_file, encoding_version)
file_details(compressed_file)

print("Compression factor: ", calculate_compression_factor(image_file, compressed_file)[0])

decompressed_image_file = "decompressed_image.bmp"
decompression_interfaces[interface](compressed_file, decompressed_image_file, encoding_version)
file_details(decompressed_image_file)

show(image_file)
show(decompressed_image_file)

File Name: Color500.bmp
File Size: 750054 bytes
File Name: compressed_file.bin
File Size: 938448 bytes
Compression factor:  0.8031622423405452
File Name: decompressed_image.bmp
File Size: 750054 bytes
